# <font color="blue">Django course, part 2:</font>

In the first part, we built the data models and the admin interface and so on. In this part, we are going to build a RESTful API for our application.

#### Setting up the project:

In DataGrip, we will create a new dabase called "storeonline". To do so, open a new Query Console, and type:

> CREATE DATABASE storeonline

So, we need to open our project and run our migrations. Then, Django will automatically create our database tables for us. Use the attached Zip file where you can find all the source codes and database seed file. 

Press "ctrl + t" and jump to the DATABASE section of the settings module. Change 'NAME': 'storeonline'. Save the changes and open the terminal:

$ pipenv install

to install all the dependencies. 

$ pipenv shell

$ python manage.py migrate

Back to DataGrip, close all the sessions, open a new Query console for storeonline. Back to zip file, go to the data folder and open the seed file in the DataGrip. Then, from the session drop-down (top right), select console_23, and in schema, select storeonline. Then, select all the statements and run them. Final step is to run from the webserver:

$ python manage.py runserver

For our new inserted database, create a new supperuser

$ python manage.py createsuperuser

## <font color="blue">2-Building RESTfull APIs</font>

#### What are RESTful APIs?

We need a way to expose our data to clients like a web app running inside a browser or a mobile app running on a mobile device. And this is where APIs (Application Programming Interface) come to the picture. Building an API is essentially building an interface that client apps can use to get or save the data. APIs are going to have a bunch of endpoints for different purposes. So, we can have an end point for getting the list of products as well as creating, updating, and deleting them. We can have other endpoints for managing our shopping carts, orders, and so on. Client apps can send requests to these endpoints and get or save products, orders, shopping carts, etc.. 

What is RESTful API?

REST is short for "Representational State Transfer" and stands for defining a bunch of ruls for clients and servers to communicate over the web. following these ruls help us build sysems that are fast, scalable, relaiable, easy to understand, and easy to change. An API that confirm these ruls called RESTful. There are three concepts that everyone shoul know: resources, resource representations, and HTTP methods.

#### Resources:

Resources in a RESTful API is objects in our application like Product, Cart, Collection. These resources are avalilable on the web and client applicaton can access them using URLs (Uniform Resource Locator). So, URL is way to locate a resource on the web. It is basically a web address. Assuming our website is hosted, for example, at http://rezabuy.com, we can access all products in  http://rezabuy.com/products or access an individual product http://rezabuy.com/products/1. A resource can contain other resources like reviews http://rezabuy.com/products/1/reviews or for an especific review http://rezabuy.com/products/1/reviews/1. As a general rule of thumb, we should not nest resources too deep, say two rows is fine, but beyond that the URL can get complex and unmanagable. The above pattern or consistency across these URLs is one of the attributes of RESTful APIs. So, if we simply follow this pattern are API will be familiar and easy to understand to others. 

#### Resource representations:

We can identify a resource using it's URL. When we hit that URL, the server returns the resource in a certain format or representation. Different way of representations are HTML, XML (an old format for representing data), or JSON (a modern format that replaces XML). None of these are an internal representation of resources on the server. In other words, on the server, we identify a resource like Product, using an object or an instance of a python class. But, we return this to the client, we represent it as a HTML, XML, or JSON. Because these are the formats that clients can understand. REST does not dictate any format or representation we should use. JSON stands for JavaScripts Object Notation. Here you see a JSON representation of a product that uses a dictionary (key-value pairs) where keis are always string format (should be surrounded by double quote) but the value can be any format like string, number, boolan, an another object (in the curly braces {}), array (like []) and so on:

In [ ]:
{
    "name":"Reza",
    "age": 37,
    "is_online": True,
    "employere":{},
    "interests":[]
}

#### HTTP methods:

when building RESTful APIs, we expose one or more endpoints for clients. each endpoind may support various kinds of operations. So, some endpoints may only allow reading data, while others may allow modifying data as well. This is where HTTP method comes to the picture. Using HTTP methods, the client can tell the server what it wants to do with the resource. So, HTTP defines method like "GET" for getting a resource or a collection of resources, "POST" for creating a resource, "PUT" for updating the resource, "PATCH" for updating a part of the resource like a subset of a properties, and "DELETE" for deleting a resource. For example, we are going to create a product. In our client app, we send a "POST /product" request to the products endpoint. So, the server knows we want to create a product. But, where is the product we want to create? We insert this product like a JSON object in the body of the request:

> {
> "title": "...",
> "price": 10
>}

The server will read this product and create it. Now, let's we want to update our product. First, we need to ask we want to update all the properties or just a subset of that. If we want to update all properties, we sould send a "PUT" request. Otherwise, we should send a "PATCH" request to the related URL (like /products/1). Similar to creating a product, we should include a product object in the body of the request in a JSON format. So, server will extract this product from the request and update it accordingly. For deleting a product, we should send a "DELETE /product/1" request, but, here no need to send a JSON object in the body of the request.

#### Installing Django REST framework:

In terminal:

$ pipenv install djangorestframework

Now, we need to add it in the installed_apps in the settings module.

> 'rest_framework',

#### Creating API views:

Now, let's see how we can create an endpoint like "127.0.0.1:8000/store/products" and then, if we send a request to this endpoint we should see all the products in our database. Back to the project, go to th store app and open the views module. In this module, we should create a view function. for details, please look at the __writing views__ in __Django_Course1__. In store>views/py:

In [ ]:
# If you see a warnings under the django, it is because the
#current python interpreter is not the same as the one that we in virtual env
# To solve, open a new terminal, $ pipenv shell 
# after executing this command, you can see the path to our virtual env
# (C:\users\....\storeonline...\script\activae)
# Copy this path (up to ../script/) and in the view menu go to the commant palette
# and search for "python interpreter". select "the interpreter path".
# Then, paste the address and append (.../script/python3) 
# (Customize it for windows)
from django.http import HttpResponse

def product_list(request):
    return HttpResponse('ok')

The next step is to map this view function to a urlpattern. So, back to project, and create a urls module, urls.py and add:

In [ ]:
from django.urls import path
from . import views
urlpatterns = [
    path('products/', views.product_list)
]

The above urls module belong to the store app. So, we need to import it to the main urls module. Go to the storeonline folder and  

In [ ]:
urlpatterns = [
    ...
    path('store/', include('store.urls'))
]

Now, refresh the browser and see the response 'ok'. This is how we can create a view function in django. In django, we have to classes "HttpRequest and HttpResponse". REST framework also has its own "Request" and "Response" classes. First, import api_view from decorators of rest_framework. If we apply this decorator (@api_view()) to our view function, the request object will become an instance of a request class that comes with the rest_framework. So, this will replace the request object in Django with the newer request object that comes with the rest_framework that is simpler and more powerful. Similarly, we replace HttpResponse object with Response object comes with the rest_framework. So, import Response class from rest_framework:

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response

@api_view()
def product_list(request):
    return Response('ok')

With the above code, if we hit the related url one more time, we see a page called __browsable api__. This __browsable api__ makes it incredibly easy to test our api endpoints in the browser. Please note that we see this page only when we hit our endpoints in the browser. If a client app like a mobile app, hits our endpoints, it does not see the browsable api. It only sees the data in the response. Define another function:

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response

@api_view()
def product_list(request):
    return Response('ok')

@api_view
def product_detail(request, id):
    return Response(id)

Next step is to map the created view function to the urlpattern. Go to the store>urls.py:

In [ ]:
from django.urls import path
from . import views
urlpatterns = [
    path('products/', views.product_list),
    path('products/<id>', views.product_detail)
]

If we type a sting like ".../store/products/a", again our endpoint still works and we get a response that is not true. So we can use a convertor like int to this parameter:

In [ ]:
from django.urls import path
from . import views
urlpatterns = [
    path('products/', views.product_list),
    path('products/<int:id>', views.product_detail)
]

#### Creating serializers:

We already created an api view. Now, instsead of including 'ok' as a response, we want to include a list of products. So, we need a way to convert product objects to JSON objects. In the Django rest framework, we have a class called "JSONRender". This class has a method called "render(dict)" that takes a dictionary object and returns a JSON object. So, if we convert product object to a python dictionary, we can pass it to this method and get a JSON object. And this is where serializer comes to the picture.

A serializer is an object that knows how to convert a model instance like a product object to a python dictionary. To create a serializer, back to the project and create a file in store folder called "serializer.py". In the serializer.py import serializer module from rest_framework and create a class called "ProductSerializer()". If you go to www.django-rest-framework.org under "API Guide>serializer fields", you can see all types of fields available to us like BooleanField, StringField, CharField, and so on. All these fields have a bunch of "core arguments" that are available like read-only, write-only, required, default, etc.. 

In [ ]:
from rest_framework import serializers

class ProductSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    title = serializers.CharField(max_length=255)
    unit_price = serializers.DecimalField(max_digits=6, decimal_places=2)

By having the serializer, we can use it to convert product object to a python dictionary.

#### Serializing objects:

Let's see how we can use serializer to convert a product object with a JSON object and include it in the response. First import Product class and ProductSerializer. Then, edit the product_detail() and create product and serializer objects. By creating the serializer object, this serializer will convert our product object to a dictionary. We can get that dictionary from "serializer.data". In this case, Django will create a JSON render object and give it this dictionary, the JSON render will convert that dictionary to a JSON object and the JSON object will end up in the Response. All this procedure is hidden from us!  

In [ ]:
from .models import Product
from .serializer import ProductSerializer

@api_view()
def product_detail(request, id):
    product = Product.objects.get(pk=id)
    serializer = ProductSerializer(product)
    return Response(serializer.data)

Now, hit the browser to see the JSON data of product 1 as an example. Note that the value of the unit_price key rendered as a string. In fact, this is one of the default settings in Django rest framework. But, we can easily override it. To do so, back to the project, and go to the storeonilne>settings.py. At the end of this module, we define a new setting:

In [ ]:
REST_FRAMEWORK = {
    'COERCE_DECIMAL_TO_STRING': False
}

Now, if we refresh the page, you see that unit_price is rendered as a decimal value. What if we ask for a product that does not exist, we get an exception. One of the conventions of the RESTful API is that if an object does not exist we should return a response and the status of the response should be __404__. So, __404__ is a standard Http code that means not found. So, we need to wrap the product_detail() function inside a try block:

In [ ]:
@api_view()
def product_detail(request, id):
    try:
        product = Product.objects.get(pk=1)
        serializer = ProductSerializer(product)
        return Response(serializer.data)
    except Product.DoesNotExist:
        return Response(status=404)

__404__ is a famus code, but generally speaking, we should avoid magic numbers like __404__. It is better to use constants since it makes our code more readable. Here we need to import status module that have a bunch of constants for various Http status codes:

In [ ]:
from rest_framework import status

@api_view()
def product_detail(request, id):
    try:
        product = Product.objects.get(pk=1)
        serializer = ProductSerializer(product)
        return Response(id)
    except Product.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

This is the way of handling the situation that the product does not exist. Repeating this try-except block is time consuming. This is where we use Django shortcuts. To do so, we need to import "get_object_or_404" class from shortcuts:

In [ ]:
from django.shortcuts import get_object_or_404
@api_view()
def product_detail(request, id):
        product = get_object_or_404(Product, pk=id)
        serializer = ProductSerializer(product)
        return Response(serializer.data)

For the product_list() class:

In [ ]:
@api_view()
def product_list(request):
    queryset = Product.objects.all()
    serializer = ProductSerializer(queryset, many=True)
    return Response(serializer.data)

#### Creating custom serializer fields:

Objects that we return from API do not necessarily have to look like the objects in our application (data models). The main reason behind this is that our data models are realy implementation details of our application. The implementation may change in the future. We may add new fields in the product class, rename existing fields, even delete them. We do not want these changes to be exposed to the clients. In fact, APIs represent the interface of our application. So, we should keep it stable as much as possible. otherwise, existing client may break. So, if we want to change our API, we have to properly study impact of the change and potentially provide different versions of APIs. Somehow we want to add a new field in the API that does not exist in the data model, say product class. For example, adding a new field called "price_with_tax". Also, if you want to use a different name from the name of a field in the product model (for example using "price" for the API instead of "unit_price", you need to add an argument indicateing "source='unit_price'"

In [ ]:
from .models import Product
from decimal import Decimal


class ProductSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    title = serializers.CharField(max_length=255)
    price = serializers.DecimalField(
        max_digits=6, decimal_places=2, source='unit_price')
    price_with_tax = serializers.SerializerMethodField(
        method_name='calculate_tax')

    def calculate_tax(self, product: Product):
        return product.unit_price * Decimal(1.1)

#### Serializing relationships:

When we return a product we can include a related object like collection. There are four ways to serialize the relationships: using primary key, string, nested object, and hyperlink. 

- __Uing primary key:__

In [ ]:
from .models import Product, Collection

class ProductSerializer(serializers.Serializer):
    ...
    collection = serializers.PrimaryKeyRelatedField(
        queryset=Collection.objects.all()
    )

- __Using string:__ 
Another way is to return collection as a string. So, we can return the name of each collection using "StringRelatedField()" method.:

In [ ]:
class ProductSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    title = serializers.CharField(max_length=255)
    ...
    collection = serializers.StringRelatedField()

But in this case, we have extra queries for each product that it is time consuming and impacts the performance. To avoid, got to the product_list() finction in the views module in the store app and include "select_related()" in queryset:

In [ ]:
@api_view()
def product_list(request):
    queryset = Product.objects.select_related('collection').all()
    ...
    return Response(serializer.data)

The serializer is as before. Just refresh the page and see you get the results instantly.

- __Nested object:__ 
a proper approach is including nested object like Collection object in the JSON data. Back to serializer, and create collection serializer:

In [ ]:
class CollectionSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    title = serializers.CharField(max_length=255)

Then, to include nested object all we have to do is to use CollectionSerializer:

In [ ]:
class ProductSerializer(serializers.Serializer):
    ...
    collection = CollectionSerializer()

- __Using hyperlink:__
Instead of using object, we can include a hypterlink to an endpoint for viewing that collection. Back to the serializer and set the collection to HypterLinkRelatedField() where we should set two arguments, one is queryset and another is view_name that is used for generating hyperlinks.

In [ ]:
class ProductSerializer(serializers.Serializer):
    ...
    collection = serializers.HyperlinkedRelatedField(
        queryset=Collection.objects.all(),
        view_name='collection-detail'
    )

Now, we need to create a view for 'collection-detail' in urls module. So in the store > urls.py

In [ ]:
urlpatterns = [
    ...
    path('collection/<int:id>', views.collection_detail,
         name='collection-detail'),
]

Go to the store > views.py and create the related view function:

In [ ]:
@api_view()
def collection_detail(request, id):
    return Response('ok')

Refresh the page and sometimes you see an exception like:

`HyperlinkedRelatedField` requires the request in the serializer context. Add `context={'request': request}` when instantiating the serializer.

In this case, we need to pass our request object to our serializer. Because the request contains informaion abour urls. So, back to views module, in the product_detail() function, set the context in a line of code that we initialize our ProductSerializer as follows:

In [ ]:
@api_view()
def product_list(request):
    ...
    serializer = ProductSerializer(
        queryset, many=True, context={'request': request})
    return Response(serializer.data)

Again you may see a different kind of error:

Could not resolve URL for hyperlinked relationship using view name "collection-detail". You may have failed to include the related model in your API, or incorrectly configured the `lookup_field` attribute on this field.


This is becuase Django rest framework expects certain convention where instead of "id" we should use "pk". Please note that you need to modify it in collection_detail() function in views module:

In [ ]:
# In store>urls.py
urlpatterns = [
    ...
    path('collection/<int:pk>', views.collection_detail,
         name='collection-detail'),
]

# In views.py
@api_view()
def collection_detail(request, pk):
    return Response('ok')

#### Model serializer:

You might think so far we are repeating ourselves. All the classes we defined in serializer are already defined in the model classes. So, if we decide to chenge or modify these fields in the future, we have to modify them in both places. The better way is using ModelSerializer. Using a ModelSerializer class we can quickly create a serializer withour all this duplicatons. 

In [ ]:
class CollectionSerializer(serializers.ModelSerializer):
    class Meta:
        model = Collection
        fields = ['id', 'title']


class ProductSerializer(serializers.ModelSerializer):
    class Meta:
        model = Product
        fields = ['id', 'title', 'price', 'price_with_tax', 'collection']
    price = serializers.DecimalField(
        max_digits=6, decimal_places=2, source='unit_price')
    price_with_tax = serializers.SerializerMethodField(
        method_name='calculate_tax')

Another but, not prefered way is using '__ all __':

In [ ]:
class ProductSerializer(serializers.ModelSerializer):
    class Meta:
        model = Product
        fields = '__all__'

#### Deserializing objects:

Deserialization happens when we receive data from clients. For example, in the client we want to create a new produt. To do this, we should send a POST request to the products endpoint (POST /product) and in the body of the request we should include product object. Now, on the server we have to read the data from the body of the request and deserialize it. So, we get a product object and store it in the database. Back to the porduct_list view function and pass some arguments to api_view(). We have to pass an argument to array of strings that specify the HTTP methods we support at this endpoint. Previously we did not have to do this because 'Get' is supported by default. But since we want to support 'POST' mothod is well, we need to explicitly pass the following array to api_view(). So, in store > views.py:

In [ ]:
@api_view(['GET', 'POST'])
def product_list(request):
    if request.method == 'GET':
        queryset = Product.objects.select_related('collection').all()
        serializer = ProductSerializer(
            queryset, many=True, context={'request': request})
        return Response(serializer.data)
    elif request.method == 'POST':
        serializer = ProductSerializer(data=request.data)
        # serializer.validated_data
        return Response('OKOKOK')

#### Data validation:

Before we can access the validated data attribute, first we have to validate the data, otherwise, we get an exception like this:

You must call `.is_valid()` before accessing `.validated_data`.

In [ ]:
@api_view(['GET', 'POST'])
def product_list(request):
    if request.method == 'GET':
        ...
    elif request.method == 'POST':
        serializer = ProductSerializer(data=request.data)
        if serializer.is_valid():
            serializer.validated_data
            return Response('OKOKOK')
        else:
            return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

There is another way to right this code that is cleaner and more consice:

In [ ]:
@api_view(['GET', 'POST'])
def product_list(request):
    if request.method == 'GET':
    ...
    elif request.method == 'POST':
        serializer = ProductSerializer(data=request.data)
        serializer.is_valid(raise_exception=True)
        serializer.validated_data
        return Response('OKOKOK')

Now, let's look at the validated data attribute. 

In [ ]:
@api_view(['GET', 'POST'])
def product_list(request):
    if request.method == 'GET':
    ...
        ...
        print(serializer.validated_data)
        return Response('OKOKOK')

And pass a valide product object to see the validated object dictionary:

> {
> "title":"a",
> "unit_price":1,
> "collection":1
>}

Another issue is validation at the object level. There are situations validating th request data in both comparing multiple fields. For exampple, think of a senario where a user registers. So, we have a user name as well as password and confirm password. We want to make sure that two fields are equal. With the current implementation we cannot achive this. To do so, back to serializer.py and define a validate method in store > serializer.py:

In [ ]:
class ProductSerializer(serializers.ModelSerializer):
    ... 
    def validate(self, data):
        if data['password'] != data['confirm_password']:
            return serializers.ValidationError('Password do not match')
        return data

#### Saving objects:

Our ProductSerializer has inherited from ModelSerializer. This ModelSerializer has a "save()" method we can use to create or update the results. Back to serializer.py:

In [ ]:
@api_view(['GET', 'POST'])
def product_list(request):
    if request.method == 'GET':
    ...
    elif request.method == 'POST':
        serializer = ProductSerializer(data=request.data)
        serializer.is_valid(raise_exception=True)
        serializer.save()
        #serializer.validated_data
        return Response('OKOKOK')

Please note that the required fields should be included in the ProductSerializer and add 'slug' and 'inventory'.